In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/DX

/content/drive/MyDrive/DX


In [1]:
import pandas as pd

## 데이터 로드

In [ ]:
df = pd.read_csv('full_data_0510.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88824 entries, 0 to 88823
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   source        88824 non-null  object
 1   search_words  88824 non-null  object
 2   url           81332 non-null  object
 3   title         30500 non-null  object
 4   contents      88824 non-null  object
dtypes: object(5)
memory usage: 3.4+ MB


In [ ]:
df.head()

,source,search_words,url,title,contents
0,블로그,아기+잠깨,https://blog.naver.com/srhymin/223091161466,NaN,저희 아기는 신생아때부터\n청각과 촉각이 예민해\n잠도 깊게 못 잘 뿐더러\n잠투정...
1,블로그,아기+잠깨,https://blog.naver.com/sukm83/223091879514,NaN,정신없는 주방한켠\n뒤죽박죽 알 수 없는 용기에\n양념과 소스가 담겨져 있기에\n주...
2,블로그,아기+잠깨,https://blog.naver.com/dldptmf5134/223096875782,NaN,안녕하세요 지유맘입니다!\n오늘은 지유가 이유식과 유아식 그 중간즈음 토핑식을 할 ...
3,블로그,아기+잠깨,https://blog.naver.com/lia1100/223095799488,NaN,안녕하세요\n라온엄마에요\n오늘은 조금 특이한 아기 반찬을 준비했어요\n방풍나물을 ...
4,블로그,아기+잠깨,https://blog.naver.com/perfect_aj/223095824533,NaN,6~12개월 아기의 1일 권장 물섭취량은\n220ml정도라고 한다.\n음 우래기는....


## 데이터 전처리하기

### 특수문자 및 숫자 제거

* 정규표현식 사용
* 알파벳, 공백, 한글, 문장부호 (느낌표, 물음표, 온점) 제외 제거

In [ ]:
import re
from tqdm import tqdm

# 제거할 문자에 대한 정규식 패턴
# (알파벳도 유지하려면 a-zA-Z 추가)
# [^\s a-zA-Z 가-힣 \! \? \.] : 공백, 영문 알파벳, 한글, !, ?, . 가 아닌 모든 문자
pattern_unwanted = re.compile(r'[^\s a-zA-Z0-9가-힣\!\?\.]')

# 연속된 공백을 하나로 줄이는 정규식 패턴
pattern_spaces = re.compile('\s+')

# .str.replace() 를 사용하여 처리 (결측값은 알아서 처리됨)
# regex=True 를 명시하여 정규식 사용을 알림
print("\n처리 중...")
# 첫 번째: 원치 않는 문자 제거 -> 공백으로 변경
processed_contents = df['contents'].str.replace(pattern_unwanted, ' ', regex=True)
# 두 번째: 연속된 공백 제거 -> 하나의 공백으로 변경
processed_contents = processed_contents.str.replace(pattern_spaces, ' ', regex=True)

# 4. 결과 덮어쓰기
df['contents'] = processed_contents

print("\n문자열 처리 후 (샘플):")
print(df['contents'].head())


처리 중...

문자열 처리 후 (샘플):
0    저희 아기는 신생아때부터 청각과 촉각이 예민해 잠도 깊게 못 잘 뿐더러 잠투정이 정...
1    정신없는 주방한켠 뒤죽박죽 알 수 없는 용기에 양념과 소스가 담겨져 있기에 주방이 ...
2    안녕하세요 지유맘입니다! 오늘은 지유가 이유식과 유아식 그 중간즈음 토핑식을 할 때...
3    안녕하세요 라온엄마에요 오늘은 조금 특이한 아기 반찬을 준비했어요 방풍나물을 이용한...
4    6 12개월 아기의 1일 권장 물섭취량은 220ml정도라고 한다. 음 우래기는.. ...
Name: contents, dtype: object


### 의미없는 짧은 글 제거

* 30 글자 이상만 사용

In [ ]:
# 30 글자 미만은 지워주기
tqdm.pandas()

df = df[df['contents'].progress_apply(lambda x : len(x) > 30)]
len(df)

100%|██████████| 88824/88824 [00:00<00:00, 407819.86it/s]


88766

In [ ]:
df = df.reset_index(drop = True)
df

,source,search_words,url,title,contents
0,블로그,아기+잠깨,https://blog.naver.com/srhymin/223091161466,NaN,저희 아기는 신생아때부터 청각과 촉각이 예민해 잠도 깊게 못 잘 뿐더러 잠투정이 정...
1,블로그,아기+잠깨,https://blog.naver.com/sukm83/223091879514,NaN,정신없는 주방한켠 뒤죽박죽 알 수 없는 용기에 양념과 소스가 담겨져 있기에 주방이 ...
2,블로그,아기+잠깨,https://blog.naver.com/dldptmf5134/223096875782,NaN,안녕하세요 지유맘입니다! 오늘은 지유가 이유식과 유아식 그 중간즈음 토핑식을 할 때...
3,블로그,아기+잠깨,https://blog.naver.com/lia1100/223095799488,NaN,안녕하세요 라온엄마에요 오늘은 조금 특이한 아기 반찬을 준비했어요 방풍나물을 이용한...
4,블로그,아기+잠깨,https://blog.naver.com/perfect_aj/223095824533,NaN,6 12개월 아기의 1일 권장 물섭취량은 220ml정도라고 한다. 음 우래기는.. ...
...,...,...,...,...,...
88761,블라인드,낮잠+육아,NaN,NaN,애기는 7개월 됐고 와이프가 육아휴직 중이야. 장모님댁 근처 걸어서 10분 거리에 ...
88762,블라인드,낮잠+육아,NaN,NaN,근데 난 이미 육아 2시간째라 곧 낮잠 1 재우고 걍 옆에서 같이 잘건데 . 몇번이...
88763,블라인드,낮잠+육아,NaN,NaN,무슨 오전 낮잠을 두시간이나 이상하게 아빠가 있는 주말이면 낮잠도 잘자고 더 잘 먹...
88764,블라인드,낮잠+육아,NaN,NaN,복직해야해서 보내는거 제외하고 돌쯤아기 육아하는 사람들요 주변에 전업이거나 육휴 계...


### 정제한 df 저장

In [ ]:
df.to_csv('cleaned_full_data_0510.csv', encoding='utf-8-sig')

In [ ]:
df = pd.read_csv('./cleaned_full_data_0510.csv', encoding='utf-8-sig')

## 불용어 제거 및 데이터 형태소 분리

In [ ]:
# Konlpy 설치
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

--2025-05-10 10:19:13--  https://www.dropbox.com/s/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/g8l6dlu5a3ebovcdymluq/mecab-0.996-ko-0.9.2.tar.gz?rlkey=xlooryouhr1l2ufqbt93ohfvy&dl=1 [following]
--2025-05-10 10:19:13--  https://www.dropbox.com/scl/fi/g8l6dlu5a3ebovcdymluq/mecab-0.996-ko-0.9.2.tar.gz?rlkey=xlooryouhr1l2ufqbt93ohfvy&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7e104c4f11d2386f7b4db78658.dl.dropboxusercontent.com/cd/0/inline/CpYxFVSs652uUa2lDM_zmqPD0y3ByjTgfeB5rG8tHVTN8_vJS6EXKVGkN73uuqxrny0PINUojQatKnIApRYskNbKiCwlZE9wQuXdtc-WwVWQCFF_Q5THT74jE_UqNslWzSbvwCTZNH4542YfLWmS1gPT/file?dl=1# [following]
--2025-05-10 10:19:13--  https://uc7

### 불용어적용 및 형태소 분리
* 클러스터링 수업 때 사용했던 한국어 불용어 파일 사용 (ko-stopwords.csv)
* 627 개의 stopwords (워드 클라우드 참고해서 추가)

In [19]:
stopwords = pd.read_csv('./dx_stopwords.csv', encoding='utf-8')
stopwords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627 entries, 0 to 626
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   stopwords  627 non-null    object
dtypes: object(1)
memory usage: 5.0+ KB


In [20]:
additional_stopwords_list = ['생각', '사실', '참깨', '통깨', '요리', '반찬']
new_stopwords_to_add_df = pd.DataFrame(additional_stopwords_list, columns=['stopwords'])
updated_stopwords_df = pd.concat([stopwords, new_stopwords_to_add_df], ignore_index=True)
updated_stopwords_df = updated_stopwords_df.drop_duplicates(subset=['stopwords'], keep='first').reset_index(drop=True)
stopwords = updated_stopwords_df
stopwords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 625 entries, 0 to 624
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   stopwords  625 non-null    object
dtypes: object(1)
memory usage: 5.0+ KB


### 데이터 프레임에 추가

* 불용어 처리
* 'Noun'만 사용함
* 형태소 분리한 내용을 새로운 변수 tagged_contents 로 저장

In [21]:
from tqdm import tqdm
tqdm.pandas()
from konlpy.tag import Okt

okt = Okt()

def pos_tagging (string) :
    posword = okt.pos(string, stem = True, norm = True)
    tagged_pos = [word for word, tag in posword if word not in stopwords if tag in {'Noun'}] # 명사만 사용
    return tagged_pos

df['tagged_contents'] = df['contents'].progress_apply(lambda x : pos_tagging(x))
df.head()

100%|██████████| 88766/88766 [9:18:54<00:00,  2.65it/s]


,Unnamed: 0,source,search_words,url,title,contents,tagged_contents
0,0,블로그,아기+잠깨,https://blog.naver.com/srhymin/223091161466,NaN,저희 아기는 신생아때부터 청각과 촉각이 예민해 잠도 깊게 못 잘 뿐더러 잠투정이 정...,"[저희, 아기, 신생아, 때, 청각, 촉각, 잠도, 못, 뿐더러, 잠투, 정이, 정..."
1,1,블로그,아기+잠깨,https://blog.naver.com/sukm83/223091879514,NaN,정신없는 주방한켠 뒤죽박죽 알 수 없는 용기에 양념과 소스가 담겨져 있기에 주방이 ...,"[정신, 방한, 뒤죽박죽, 알, 수, 용기, 양념, 소스, 주방, 이번, 양념, 통..."
2,2,블로그,아기+잠깨,https://blog.naver.com/dldptmf5134/223096875782,NaN,안녕하세요 지유맘입니다! 오늘은 지유가 이유식과 유아식 그 중간즈음 토핑식을 할 때...,"[지유, 맘, 오늘, 지유, 이유식, 유아식, 그, 중간, 즈음, 토핑식, 때, 정..."
3,3,블로그,아기+잠깨,https://blog.naver.com/lia1100/223095799488,NaN,안녕하세요 라온엄마에요 오늘은 조금 특이한 아기 반찬을 준비했어요 방풍나물을 이용한...,"[엄마, 오늘, 조금, 아기, 반찬, 준비, 방, 나물, 이용, 방, 나물, 된장,..."
4,4,블로그,아기+잠깨,https://blog.naver.com/perfect_aj/223095824533,NaN,6 12개월 아기의 1일 권장 물섭취량은 220ml정도라고 한다. 음 우래기는.. ...,"[개월, 아기, 권장, 물섭취량, 정도, 음, 우리, 애기, 생각, 물, 물병, 번..."


## 데이터 저장하기

In [22]:
import pickle

with open('./tagged_contents_df_0510.pkl','wb') as f:
    pickle.dump(df, f)

In [23]:
stopwords.to_csv('dx_stopwords_custom_0510.csv', encoding='utf-8-sig')